In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

# 모델 및 토크나이저 로드
model_name = "unsloth/Meta-Llama-3.1-8B"  # 사용할 모델 이름
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

In [1]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 101,
    use_rslora = False,
    loftq_config = None,
)


NameError: name 'FastLanguageModel' is not defined

In [ ]:
# Alpaca 포맷 정의
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# EOS 토큰 설정
EOS_TOKEN = tokenizer.eos_token

# Define PDF processing functions
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """Extract text from PDF and split into chunks."""
    import fitz  # PyMuPDF
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.schema import Document
    
    # Open PDF and extract text
    doc = fitz.open(file_path)
    text = ''
    for page in doc:
        text += page.get_text()

    # Split text into chunks
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunk_temp = splitter.split_text(text)
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks

def create_vector_db(chunks, model_path="intfloat/multilingual-e5-base"):
    """Create a FAISS vector database from chunks."""
    from langchain.embeddings import HuggingFaceEmbeddings
    from langchain.vectorstores import FAISS

    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
    )
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """Normalize path for Unicode compatibility."""
    import unicodedata
    return unicodedata.normalize('NFC', path)

def process_pdfs_from_dataframe(df, base_directory):
    """Process PDFs listed in DataFrame and create vector DBs and retrievers."""
    from tqdm import tqdm

    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./')))
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 8})
        
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases
    
# Load and preprocess training data
base_directory = './'  # Your base directory for PDFs
df = pd.read_csv('/home/jovyan/temp/open/train.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

In [ ]:
# Define data formatting function
def formatting_prompts_func(examples):
    questions = examples["Question"]
    answers = examples["Answer"]
    texts = []
    for question, answer in zip(questions, answers):
        text = alpaca_prompt.format(question, "", answer) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Load dataset
dataset = load_dataset('csv', data_files="/home/jovyan/temp/open/train.csv", split='train')

# Apply formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)

# Check formatted data
print(dataset[0])

In [13]:
# 모델 훈련 설정
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # 짧은 시퀀스에 대해 훈련을 빠르게 할 수 있습니다.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,  # 최대 훈련 스텝 수
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=101,
        output_dir="outputs",
    ),
)


max_steps is given, it will override any value given in num_train_epochs


In [14]:
# 훈련 실행
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 496 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.999800
2,2.513700
3,2.705300
4,2.625100
5,2.563900
6,2.283700
7,2.210500
8,1.826000
9,1.661200
10,1.788800


In [18]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "2024년 기준으로 일반회계 특별회계 기금은 각각 몇개인가?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n2024년 기준으로 일반회계 특별회계 기금은 각각 몇개인가?\n\n### Input:\n\n\n### Response:\n2024년 기준으로 일반회계 특별회계 기금은 각각 17개, 11개이다.<|end_of_text|>']

In [16]:
#정답
#2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.

In [20]:
model.save_pretrained("llama3_model_파인튜닝2") # Local saving
tokenizer.save_pretrained("llama3_model_파인튜닝2")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('llama3_model_파인튜닝2/tokenizer_config.json',
 'llama3_model_파인튜닝2/special_tokens_map.json',
 'llama3_model_파인튜닝2/tokenizer.json')